In [1]:
# ------------------------------------------------------------
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 8.1 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=e319516c62819349125f3fa7227e7f9b84de2928f02d4993f1fe0607c96258fb
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [2]:
# ------------------------------------------------------------
import os
import random
import copy

import numpy as np
import pandas as pd

from lifelines import CoxPHFitter
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    average_precision_score,
)

In [3]:
# ------------------------------------------------------------
RNG_SEED = 42

def seed_all(seed: int = RNG_SEED) -> None:
    """
    Set random seeds for Python, NumPy, and OS hashing
    to make results reproducible.
    """
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

seed_all(RNG_SEED)

In [4]:
# ------------------------------------------------------------
Data001 = pd.read_csv(
    "/content/drive/MyDrive/Paper(2025Dec)_SimulatedCVD/Data001_ReadyForCoxPH.csv"
)

# Drop index-like columns if present
for col in ["Unnamed: 0", "index"]:
    if col in Data001.columns:
        Data001 = Data001.drop(columns=[col])

df_raw = copy.copy(Data001)


In [5]:
# ------------------------------------------------------------
train_df, test_df = train_test_split(
    df_raw,
    test_size=0.2,
    random_state=RNG_SEED,
    stratify=df_raw["cvd_event"],
)

print(f"Train size: {train_df.shape[0]}, Test size: {test_df.shape[0]}")


Train size: 40000, Test size: 10000


In [7]:
# ------------------------------------------------------------
def build_design_matrix(df: pd.DataFrame, drop_cols=None):
    df = df.copy()

    df["Age_c"] = df["Age"] - 30.0

    irsd_dummies = pd.get_dummies(
        df["IRSD_quintile"],
        prefix="irsd",
        drop_first=False
    )
    if "irsd_5" in irsd_dummies.columns:
        irsd_dummies = irsd_dummies.drop(columns=["irsd_5"])

    smoke_dummies = pd.get_dummies(
        df["smoking_status"],
        prefix="smoke",
        drop_first=False
    )
    smoke_dummies = smoke_dummies.fillna(0)
    if "smoke_non" in smoke_dummies.columns:
        smoke_dummies = smoke_dummies.drop(columns=["smoke_non"])

    for col in ["AF", "CKD", "diabetes", "cvd_event"]:
        df[col] = df[col].astype(int)

    X = pd.concat(
        [
            df[["Age_c", "AF", "CKD", "diabetes", "HbA1c", "eGFR", "SBP"]],
            smoke_dummies,
            irsd_dummies,
        ],
        axis=1
    )

    cols_to_drop = []
    if drop_cols is None:
        events = df["cvd_event"].astype(bool)

        for col in X.columns:
            v_all      = X[col].var()
            v_event    = X.loc[events, col].var() if events.sum() > 0 else 0.0
            v_nonevent = X.loc[~events, col].var() if (~events).sum() > 0 else 0.0

            if (v_all < 1e-6) or (v_event < 1e-6) or (v_nonevent < 1e-6):
                cols_to_drop.append(col)
    else:
        cols_to_drop = drop_cols

    X_reduced = X.drop(columns=cols_to_drop, errors="ignore")

    return X_reduced, cols_to_drop

In [8]:
# ------------------------------------------------------------
X_train, cols_to_drop = build_design_matrix(train_df, drop_cols=None)

cox_train_df = pd.concat(
    [
        train_df[["cvd_time", "cvd_event"]].reset_index(drop=True),
        X_train.reset_index(drop=True),
    ],
    axis=1
)

cox_train_df = cox_train_df.dropna()

y_time_train  = cox_train_df["cvd_time"].values
y_event_train = cox_train_df["cvd_event"].values.astype(int)
X_train_final = cox_train_df.drop(columns=["cvd_time", "cvd_event"])

print(f"Final training matrix shape: {X_train_final.shape}")

cph = CoxPHFitter(penalizer=0.05)
cph.fit(
    cox_train_df,
    duration_col="cvd_time",
    event_col="cvd_event",
    show_progress=True,
)

print("\n=== CoxPH summary (training) ===")
cph.print_summary()


Final training matrix shape: (40000, 13)
Iteration 1: norm_delta = 1.07e+00, step_size = 0.9500, log_lik = -16867.06666, newton_decrement = 2.64e+03, seconds_since_start = 0.4
Iteration 2: norm_delta = 6.65e-01, step_size = 0.9500, log_lik = -16167.55102, newton_decrement = 1.43e+03, seconds_since_start = 0.8
Iteration 3: norm_delta = 2.34e-01, step_size = 0.9500, log_lik = -15314.75821, newton_decrement = 2.25e+02, seconds_since_start = 1.1
Iteration 4: norm_delta = 2.46e-02, step_size = 1.0000, log_lik = -15111.86949, newton_decrement = 3.60e+00, seconds_since_start = 1.5
Iteration 5: norm_delta = 2.09e-04, step_size = 1.0000, log_lik = -15108.25879, newton_decrement = 9.62e-05, seconds_since_start = 1.9
Iteration 6: norm_delta = 1.00e-08, step_size = 1.0000, log_lik = -15108.25869, newton_decrement = 4.72e-13, seconds_since_start = 2.3
Convergence success after 6 iterations.

=== CoxPH summary (training) ===


<lifelines.CoxPHFitter: fitted with 40000 total observations, 38393 right-censored observations>
             duration col = 'cvd_time'
                event col = 'cvd_event'
                penalizer = 0.05
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 40000
number of events observed = 1607
   partial log-likelihood = -15108.26
         time fit was run = 2026-01-29 22:11:19 UTC

---
               coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                      
Age_c          0.03      1.03      0.00            0.03            0.03                1.03                1.03
AF             1.12      3.07      0.15            0.84            1.41                2.31                4.09
CKD           -0.04      0.96      0.15           -0.33            0.25                0.72                1.28
diabetes       1.42      4.14      0.05            1.31            1.53                3.72                4.61
HbA1c          0.32      1.38      0.02            0.29            0.36                1.34                1.43
eGFR          -0.02      0.98      0.00           -0.02           -0.01                0.98                0.99
SBP            0.01      1.01      0.00            0.01            0.01                1.01                1.01
smoke_current  0.16      1.18      0.05            0.06            0.27                1.06                1.31
smoke_ex       0.14      1.15      0.04            0.06            0.23                1.06                1.26
irsd_1         0.01      1.01      0.04           -0.07            0.10                0.94                1.10
irsd_2         0.05      1.05      0.05           -0.04            0.14                0.96                1.15
irsd_3         0.05      1.05      0.04           -0.03            0.12                0.97                1.13
irsd_4        -0.01      0.99      0.05           -0.10            0.08                0.91                1.09

               cmp to     z      p  -log2(p)
covariate                                   
Age_c            0.00 20.90 <0.005    319.95
AF               0.00  7.73 <0.005     46.37
CKD              0.00 -0.29   0.77      0.37
diabetes         0.00 25.88 <0.005    488.26
HbA1c            0.00 20.24 <0.005    300.29
eGFR             0.00 -6.65 <0.005     34.97
SBP              0.00 10.15 <0.005     77.97
smoke_current    0.00  3.01 <0.005      8.56
smoke_ex         0.00  3.25 <0.005      9.75
irsd_1           0.00  0.35   0.73      0.46
irsd_2           0.00  1.01   0.31      1.67
irsd_3           0.00  1.14   0.25      1.98
irsd_4           0.00 -0.16   0.87      0.20
---
Concordance = 0.87
Partial AIC = 30242.52
log-likelihood ratio test = 3517.62 on 13 df
-log2(p) of ll-ratio test = inf

In [9]:
# ------------------------------------------------------------
X_test, _ = build_design_matrix(test_df, drop_cols=cols_to_drop)

cox_test_df = pd.concat(
    [
        test_df[["cvd_time", "cvd_event"]].reset_index(drop=True),
        X_test.reset_index(drop=True),
    ],
    axis=1
)

cox_test_df = cox_test_df.dropna()

y_time_test  = cox_test_df["cvd_time"].values
y_event_test = cox_test_df["cvd_event"].values.astype(int)
X_test_final = cox_test_df.drop(columns=["cvd_time", "cvd_event"])

print(f"Final test matrix shape: {X_test_final.shape}")

train_risk = cph.predict_partial_hazard(X_train_final).values.ravel()
test_risk  = cph.predict_partial_hazard(X_test_final).values.ravel()


Final test matrix shape: (10000, 13)


In [10]:
# ------------------------------------------------------------
threshold = np.median(train_risk)

y_pred_test = (test_risk >= threshold).astype(int)

accuracy  = accuracy_score(y_event_test, y_pred_test)
precision = precision_score(y_event_test, y_pred_test, zero_division=0)
recall    = recall_score(y_event_test, y_pred_test, zero_division=0)
f1        = f1_score(y_event_test, y_pred_test, zero_division=0)

auroc = roc_auc_score(y_event_test, test_risk)
auprc = average_precision_score(y_event_test, test_risk)

print("\n=== Test-set binary discrimination ===")
print(f"Accuracy : {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"F1-score : {f1:.3f}")
print(f"AUROC    : {auroc:.3f}")
print(f"AUPRC    : {auprc:.3f}")


=== Test-set binary discrimination ===
Accuracy : 0.543
Precision: 0.076
Recall   : 0.930
F1-score : 0.141
AUROC    : 0.879
AUPRC    : 0.420
